# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способ заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [39]:
# работа sklearn с дефолтной токенизацией:

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

from IPython.display import Image
from IPython.core.display import HTML

data = pd.read_csv('labeled.csv')
train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train.comment)
vectorizer = TfidfVectorizer(min_df=10, max_df=0.3)
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)
y = train.toxic.values
y_test = test.toxic.values
clf = LogisticRegression(C=0.1, class_weight='balanced')
clf.fit(X, y)
clf.classes_
preds = clf.predict(X_test)
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.88      0.83      0.85       991
         1.0       0.67      0.75      0.70       451

    accuracy                           0.80      1442
   macro avg       0.77      0.79      0.78      1442
weighted avg       0.81      0.80      0.81      1442



In [40]:
# определяем кастомный токенизатор и используем его в sklearn:

from razdel import tokenize
def my_tokenizer(doc):
    tokens = tokenize(doc)
    return([token.text for token in tokens])

vectorizer = CountVectorizer(tokenizer=my_tokenizer)
X = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment)
X.shape, X_test.shape
y = train.toxic.values
y_test = test.toxic.values
clf = LogisticRegression(C=0.1, class_weight='balanced', max_iter=200)
clf.fit(X, y)
clf.classes_
preds = clf.predict(X_test)
print(classification_report(y_test, preds, zero_division=0))

              precision    recall  f1-score   support

         0.0       0.91      0.84      0.87       991
         1.0       0.70      0.82      0.75       451

    accuracy                           0.83      1442
   macro avg       0.80      0.83      0.81      1442
weighted avg       0.84      0.83      0.84      1442



## Задание 2 (3 балла)

Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Считать нужно в питоне. Формат итоговой таблицы может быть любым, главное, чтобы был код и можно было воспроизвести вычисления. 

In [41]:
# я на начинающем питоне, поэтому так ("цикл for это плохо, а цикл for в цикле for еще хуже")
# это список по предложениям (их 5), здесь посчитано, сколько раз встречается в каждом их них каждое слово
list_general = [[1,1,1,0,0,0],[1,1,1,0,0,0],[3,0,1,1,0,0],[1,0,0,1,1,0],[0,0,0,0,0,1]]
# считаем tf:
list_tf = []
for i in list_general:
    m = []
    for j in i:
        m.append(j/sum(i))
    list_tf.append (m)
list_tf

[[0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.0, 0.0, 0.0],
 [0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.0, 0.0, 0.0],
 [0.6, 0.0, 0.2, 0.2, 0.0, 0.0],
 [0.3333333333333333, 0.0, 0.0, 0.3333333333333333, 0.3333333333333333, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]

In [43]:
# считаем idf:
import math
spisok = [] # список количеств документов, в которых есть слово, подряд по всем словам (их 6)
for j in range (len(list_general[0])):
    d = 0 
    for i in list_general:
        if i[j]:
            d += 1
    spisok.append(d)
list_idf = [] # список idf по каждому слову
for i in spisok:
    list_idf.append(math.log(i/len(list_general)))
list_idf

[-0.2231435513142097,
 -0.916290731874155,
 -0.5108256237659907,
 -0.916290731874155,
 -1.6094379124341003,
 -1.6094379124341003]

In [44]:
# считаем tf-idf для каждого слова в каждом предложении
list_tf_idf = []
for i in list_tf:
    lst = []
    for j in range(len(list_idf)):
        lst.append(round(i[j]*list_idf[j],2))
    list_tf_idf.append(lst)
list_tf_idf

[[-0.07, -0.31, -0.17, -0.0, -0.0, -0.0],
 [-0.07, -0.31, -0.17, -0.0, -0.0, -0.0],
 [-0.13, -0.0, -0.1, -0.18, -0.0, -0.0],
 [-0.07, -0.0, -0.0, -0.31, -0.54, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0, -1.61]]

## Задание 3 (2 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к классификаторам:   
а) один должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров  
в) у классификатора должно быть задано вручную как минимум 2 параметра  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

In [ ]:
# ваш код

## *Задание 4 (2 балла)

Для классификаторов LogisticRegression, Decision Trees, Naive Bayes, Random Forest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.

In [6]:
# ваш код